In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

# Data Collection

In [2]:
url = requests.get('http://www.it.kmitl.ac.th/~teerapong/news_archive/index.html')#root URL
url_text = BeautifulSoup(url.content,'html')

In [3]:
list_path, list_month = [], [] #สร้าง list เปล่าเพื่อนำมาเก็บข้อมูลของ URL 

for i in url_text.find_all('a',href = True):
  if i['href'] != '#': 
    list_path.append('http://www.it.kmitl.ac.th/~teerapong/news_archive/' + i['href'])
    list_month.append(i.text)

In [4]:
list_category, list_month_news, list_urlnews, list_path_content = [], [], [], []

for i in list_path:
  url_month = requests.get(i)
  url_mtext = BeautifulSoup(url_month.content,'html')
  url_mtextbody = url_mtext.tbody.find_all('a',href = True) #เอาเฉพาะใน tags <tbody>
  for j in url_mtext.find_all('td', {'class':'category'}):
    if j.text.strip() != 'N/A':
        list_category.append(j.text.strip())
        list_month_news.append(list_month[list_path.index(i)])
  for k in url_mtextbody:
    list_urlnews.append('http://www.it.kmitl.ac.th/~teerapong/news_archive/' + k['href'])
    list_path_content.append(k['href'].split('.')[0])   

In [5]:
list_content ,list_header = [],[]

for i in list_urlnews:
    url_news = requests.get(i)
    content_txt = BeautifulSoup(url_news.content,'html')
    content_txt.find('p',{'class':'notice'}).decompose()
    list_header.append(content_txt.h2.text)
    test_list = []
    for j in content_txt.find_all('p'):
     test_list.append(j.text)
    list_content.append(''.join(test_list))

# Test

In [6]:
#content_txt.select("h2")[0].text

In [7]:
df = pd.DataFrame({ 'Header':list_header,'Content':list_content, 'Category':list_category, 'Month':list_month_news, 'Path':list_path_content})

In [28]:
df2 = pd.DataFrame({ 'Header':list_header,'Content':list_content,'Category':list_category})

In [50]:
df2 = df2.drop_duplicates(subset=['Header'])

In [51]:
df2 = df2.drop_duplicates(subset=['Content'])

In [52]:
df2['Header'].value_counts()

Davenport dismantles young rival     1
Kewell waits as injury drags on      1
Time to get tough on friendlies?     1
Mirza shocks Kuznetsova              1
Computer grid to help the world      1
                                    ..
Solutions to net security fears      1
Sales 'fail to boost High Street'    1
Car giant hit by Mercedes slump      1
Teenager Tait picked for England     1
Britons fed up with net service      1
Name: Header, Length: 1339, dtype: int64

In [53]:
df2 = df2.reset_index(drop=True)

In [55]:
df2

,Header,Content,Category
0,21st-Century Sports: How Digital Technology Is...,The sporting industry has come a long way sinc...,technology
1,Asian quake hits European shares,Asian quake hits European sharesShares in Euro...,business
2,BT offers free net phone calls,BT is offering customers free internet telepho...,technology
3,Barclays shares up on merger talk,Barclays shares up on merger talkShares in UK ...,business
4,Barkley fit for match in Ireland,England centre Olly Barkley has been passed fi...,sport
...,...,...,...
1334,Woodward eyes Brennan for Lions,Woodward eyes Brennan for LionsToulouse's form...,sport
1335,WorldCom trial starts in New York,"The trial of Bernie Ebbers, former chief execu...",business
1336,Yukos accused of lying to court,Yukos accused of lying to courtRussian oil fir...,business
1337,Yukos drops banks from court bid,Russian oil company Yukos has dropped the thre...,business


In [64]:
df2.duplicated().sum()

0

In [66]:
np.savetxt(r'C:\Users\MARK\Desktop\project\category.txt', df2['Category'].values, fmt="%s",encoding='utf-8')

In [67]:
np.savetxt(r'C:\Users\MARK\Desktop\project\textbody.txt', df2['Content'].values, fmt="%s",encoding='utf-8')

In [68]:
np.savetxt(r'C:\Users\MARK\Desktop\project\headerbody.txt', df2[['Header', 'Content']].values, fmt="%s",encoding='utf-8')